In [42]:
#Librerias a usar
import pandas as pd
import psycopg2

In [43]:
# Conexión a la base de datos PostgreSQL
conexion = psycopg2.connect(
    host="localhost",
    port="5432",
    database="CIBERPOL",
    user="postgres",
    password="password"
)
cursor = conexion.cursor()
print("Conexión establecida correctamente")

Conexión establecida correctamente


In [44]:
archivo_excel = 'Policias.xlsx'

# Especifica los tipos de datos para las columnas
dtype = {
    'C.C': str,            # Cedula como cadena (str) para mantener los ceros a la izquierda
    'TELEFONO_AGENTE': str,          # Numero de telefono como cadena (str)
    'TELEFONOP': str,          # Numero de telefono como cadena (str)
}

# Lee el archivo Excel con los tipos de datos especificados
df = pd.read_excel(archivo_excel, sheet_name='Nombres', engine='openpyxl', dtype=dtype)


In [45]:
# Tratar valores nulos en el DataFrame de pandas
df = df.where(pd.notna(df), None)

In [46]:
df.head()

,DIRECCION-UNIDAD,GRADO,APELLIDOS Y NOMBRES,C.C,ZONA,SUB ZONA,DISTRITO_CANTON,PASE DNTH,FUNCION DESIGNADA,NOVEDAD,...,MASCULINO_FEMENINO,TELEFONO_AGENTE,CORREO ELECTRONICO,NOMBRES Y APELLIDOS,PARENTESCO,TELEFONOP,TERNO,CAMISA,CALZADO,CABEZA
0,CIBERPOL,TCNL.,GARCÍA CATAÑA HÉCTOR GONZALO,1707489660,ZONA 9,DMQ,EUGENIO ESPEJO / QUITO,NOPERA-DGIN-UN-CIBER-JEFA-,JEFE NACIONAL DE CIBERDELITO,None,...,MASCULINO,0987874039,hector. garcia@policia.gob.ec,ROSSIGNOLI LATORRE MARIA MERCEDES,ESPOSA,23431638,42,42,41,57
1,CIBERPOL,MAYR.,RAMOS BERMEO CRISTINA ELIZABETH,1708779739,ZONA 9,DMQ,EUGENIO ESPEJO / QUITO,NOPERA-DGIN-UN-CIBER-COP-,JEFE DE COORDINACIÓN OPERACIONAL DE INVESTIGACIÓN,None,...,FEMENINO,0999082657,crstina.ramos@policia.gob.ec,ALEXIS SOLANO DE LA SALA,ESPOSO,0999397518,36,36,36,54
2,CIBERPOL,CPTN.,RIVERA FIERRO ESTEBAN ALEJANDRO,0401512371,ZONA 9,DMQ,EUGENIO ESPEJO / QUITO,NOPERA-DGIN-UN-CIBER-COP-,JEFE DE COORDINACIÓN OPERACIONAL DE INVESTIGACIÓN,None,...,MASCULINO,0969079035,esteban.rivera@policia.gob.ec,CEVALLOS ZAPATA PRISCILA ESTEFANIA,ESPOSA,2409059,40,40,41,59
3,CIBERPOL,TNTE.,MUÑOZ NAVARRETE RICARDO MARTÍN,1721535662,ZONA 9,DMQ,EUGENIO ESPEJO / QUITO,NOPERA-DGIN-UN-CIBER-OPER-,POLICÍA INVESTIGATIVO 2,None,...,MASCULINO,0998718550,ricardo.muñoz@policia.gob.ec,GUSTAVO MUÑOZ,PADRE,023431475,40,40,40,57
4,CIBERPOL,TNTE.,FLORES ORTEGA CHRISTIAN ANDRÉS,1725479628,ZONA 9,DMQ,EUGENIO ESPEJO / QUITO,NOPERA-DGIN-UN-CIBER-OPER-,POLICÍA INVESTIGATIVO 2,None,...,MASCULINO,0995512895,christian.flores@policia.gob.ec,MARCELO FLORES,PADRE,0997762010,38,38,39,59


In [47]:
print(df.columns)

Index(['DIRECCION-UNIDAD', 'GRADO', 'APELLIDOS Y NOMBRES', 'C.C', 'ZONA ',
       'SUB ZONA ', 'DISTRITO_CANTON', 'PASE DNTH', 'FUNCION DESIGNADA',
       'NOVEDAD', 'DETALLE', 'DOCUMENTO', 'TITULO ACADEMICO',
       'IDIOMA EXTRANJERO QUE DOMINE ', 'TIPO DE LICENCIA', 'RESIDENCIA',
       'ESTADO CIVIL', 'F. DE NACIMIENTO ', 'MASCULINO_FEMENINO',
       'TELEFONO_AGENTE', 'CORREO ELECTRONICO', 'NOMBRES Y APELLIDOS',
       'PARENTESCO ', 'TELEFONOP', 'TERNO', 'CAMISA', 'CALZADO', 'CABEZA'],
      dtype='object')


In [52]:
columnas_Policias = ['DIRECCION-UNIDAD', 'GRADO', 'APELLIDOS Y NOMBRES', 'C.C', 'ZONA ',
       'SUB ZONA ', 'DISTRITO_CANTON', 'PASE DNTH', 'FUNCION DESIGNADA',
       'NOVEDAD', 'DETALLE', 'DOCUMENTO', 'TITULO ACADEMICO',
       'IDIOMA EXTRANJERO QUE DOMINE ', 'TIPO DE LICENCIA', 'RESIDENCIA',
       'ESTADO CIVIL', 'F. DE NACIMIENTO ', 'MASCULINO_FEMENINO', 'TELEFONO_AGENTE',
       'CORREO ELECTRONICO', 'NOMBRES Y APELLIDOS', 'PARENTESCO ', 'TELEFONOP',
       'TERNO', 'CAMISA', 'CALZADO', 'CABEZA']


In [53]:
# Iterar sobre las filas del DataFrame e insertar en la base de datos
for indice, fila in df[columnas_Policias].iterrows():
    try:
        # Reemplazar los 'None' o celdas vacías por NULL
        valores = tuple(None if pd.isna(valor) else valor for valor in fila)

        # Mandar a los Campos Específicos de la Tabla en PostgreSQL
        consulta = """
            INSERT INTO "Agente" (
                "Direcion_Unidad", "Grado", "Apellido_Nombre", "Cedula", 
                "Zona", "SubZona", "Distrito_Canton", "PaseDNTH", "Funcion", 
                "Novedad", "Detalle", "Documento", "Titulo", "IdiomaExtranjero", 
                "Licencia", "Residencia", "Estado_Civil", "FechaNacimiento", 
                "Genero", "Telefono", "Email", "NombresFamiliar", "Parentesco", 
                "TelefonoFamiliar", "Terno", "Camisa", "Calzado", "Cabeza"
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                      %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(consulta, valores)
        conexion.commit()  # Confirmar la transacción

    except Exception as e:
        print("Error al insertar en la fila {}: {}".format(indice, e))

Error al insertar en la fila 0: cursor already closed
Error al insertar en la fila 1: cursor already closed
Error al insertar en la fila 2: cursor already closed
Error al insertar en la fila 3: cursor already closed
Error al insertar en la fila 4: cursor already closed
Error al insertar en la fila 5: cursor already closed
Error al insertar en la fila 6: cursor already closed
Error al insertar en la fila 7: cursor already closed
Error al insertar en la fila 8: cursor already closed
Error al insertar en la fila 9: cursor already closed
Error al insertar en la fila 10: cursor already closed
Error al insertar en la fila 11: cursor already closed
Error al insertar en la fila 12: cursor already closed
Error al insertar en la fila 13: cursor already closed
Error al insertar en la fila 14: cursor already closed
Error al insertar en la fila 15: cursor already closed
Error al insertar en la fila 16: cursor already closed
Error al insertar en la fila 17: cursor already closed
Error al insertar en

In [ ]:
# Cerrar el cursor y la conexión
try:
    conexion.close()
    print("Conexión cerrada correctamente")
except Exception as e:
    print("Error al cerrar la conexión:", e)